In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer

import re

path_test_txt = '../data/tweeter/test.txt'
path_train_txt = '../data/tweeter/train.txt'
path_val_txt = '../data/tweeter/val.txt'

path_glove_txt = '../data/tweeter/glove.6B.200d.txt'

path_model_keras = '../models/t2es_rnn.keras'
path_model_saved = '../models/t2es_rnn'
path_model_tflite = '../models/t2es_rnn.tflite'

In [2]:
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

lemmatizer= WordNetLemmatizer()

nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\trido\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\trido\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\trido\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
def try_rm_dupl_df(df: pd.DataFrame):

    index = df[df.duplicated() == True].index
    df.drop(index, axis = 0, inplace = True)
    df.reset_index(inplace=True, drop = True)

    #removing duplicated text
    index = df[df['Text'].duplicated() == True].index
    df.drop(index, axis = 0, inplace = True)
    df.reset_index(inplace=True, drop = True)

In [4]:
df_train = pd.read_csv(path_train_txt, names=['Text', 'Emotion'], sep=';')
df_val = pd.read_csv(path_val_txt, names=['Text', 'Emotion'], sep=';')
df_test = pd.read_csv(path_test_txt, names=['Text', 'Emotion'], sep=';')

try_rm_dupl_df(df_train)
try_rm_dupl_df(df_val)
try_rm_dupl_df(df_test)

In [5]:
def dataframe_difference(df1, df2, which=None):
    """Find rows which are different between two DataFrames."""

    # Combine the two DataFrames using a merge operation, with the
    # indicator parameter set to True. This adds a column called _merge
    # to the resulting DataFrame, which indicates the source of each row.
    comparison_df = df1.merge(
        df2,
        indicator=True,
        how='outer'
    )

    # Filter the merged DataFrame based on the value of _merge. If which
    # is not specified, return all rows where _merge is not 'both'.
    # Otherwise, return all rows where _merge has the specified value
    if which is None:
        diff_df = comparison_df[comparison_df['_merge'] != 'both']
    else:
        diff_df = comparison_df[comparison_df['_merge'] == which]

    # Return the filtered DataFrame
    return diff_df

In [6]:
dataframe_difference(df_train, df_test, which='both')

,Text,Emotion,_merge


In [7]:
dataframe_difference(df_train, df_val, which='both')

,Text,Emotion,_merge


In [8]:
dataframe_difference(df_val, df_test, which='both')

,Text,Emotion,_merge


In [9]:
def lemmatization(text):
    lemmatizer= WordNetLemmatizer()

    text = text.split()

    text=[lemmatizer.lemmatize(y) for y in text]

    return " " .join(text)

def remove_stop_words(text):

    Text=[i for i in str(text).split() if i not in stop_words]
    return " ".join(Text)

def Removing_numbers(text):
    text=''.join([i for i in text if not i.isdigit()])
    return text

def lower_case(text):

    text = text.split()

    text=[y.lower() for y in text]

    return " " .join(text)

def Removing_punctuations(text):
    ## Remove punctuations
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)
    text = text.replace('؛',"", )

    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)
    text =  " ".join(text.split())
    return text.strip()

def Removing_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_small_sentences(df):
    for i in range(len(df)):
        if len(df.text.iloc[i].split()) < 3:
            df.text.iloc[i] = np.nan

def normalized_sentence(sentence):
    sentence= lower_case(sentence)
    sentence= remove_stop_words(sentence)
    sentence= Removing_numbers(sentence)
    sentence= Removing_punctuations(sentence)
    sentence= Removing_urls(sentence)
    sentence= lemmatization(sentence)
    return sentence

def normalize_text(df):
    df.Text=df.Text.apply(lambda text : lower_case(text))
    df.Text=df.Text.apply(lambda text : remove_stop_words(text))
    df.Text=df.Text.apply(lambda text : Removing_numbers(text))
    df.Text=df.Text.apply(lambda text : Removing_punctuations(text))
    df.Text=df.Text.apply(lambda text : Removing_urls(text))
    df.Text=df.Text.apply(lambda text : lemmatization(text))
    return df

In [10]:
df_train= normalize_text(df_train)
df_test= normalize_text(df_test)
df_val= normalize_text(df_val)

In [11]:
#Splitting the text from the labels
X_train = df_train['Text']
y_train = df_train['Emotion']

X_test = df_test['Text']
y_test = df_test['Emotion']

X_val = df_val['Text']
y_val = df_val['Emotion']

# Encode labels
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)
y_val = le.transform(y_val)

#print the labels after encoding
print(set(y_train))

#Convert the class vector (integers) to binary class matrix
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)

# Tokenize words
tokenizer = Tokenizer(oov_token='UNK')
tokenizer.fit_on_texts(pd.concat([X_train, X_test], axis=0))

le.classes_

{0, 1, 2, 3, 4, 5}


array(['anger', 'fear', 'joy', 'love', 'sadness', 'surprise'],
      dtype=object)

In [12]:
# test

#document count: A dictionary of words and how many documents each appeared in.
#in this dataset the output will be the number of rows
print(tokenizer.document_count)
#print the index of a single word
print(tokenizer.word_index['towards'])
#converting a single sentence to list of indexes
print(tokenizer.texts_to_sequences(X_train[0].split()))
#the sentence contains three words and the size of the vocabulary is 14325
print(tokenizer.texts_to_matrix(X_train[0].split()).shape)

17969
220
[[53], [2], [531]]
(3, 14325)


In [13]:
maxlen = max([len(t) for t in df_train['Text']])
print(maxlen)

sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_test = tokenizer.texts_to_sequences(X_test)
sequences_val = tokenizer.texts_to_sequences(X_val)

X_train = pad_sequences(sequences_train, maxlen=229, truncating='pre')
X_test = pad_sequences(sequences_test, maxlen=229, truncating='pre')
X_val = pad_sequences(sequences_val, maxlen=229, truncating='pre')

vocabSize = len(tokenizer.index_word) + 1
print(f"Vocabulary size = {vocabSize}")

229
Vocabulary size = 14325


In [14]:
# test

#before
print(sequences_train[0])
#after
print(X_train[0])

[53, 2, 531]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0  53   2 531]


In [16]:
model = tf.keras.models.load_model(path_model_saved)

def predict(sentence):
    print(sentence)
    sentence = normalized_sentence(sentence)
    sentence = tokenizer.texts_to_sequences([sentence])
    sentence = pad_sequences(sentence, maxlen=229, truncating='pre')
    result = le.inverse_transform(np.argmax(model.predict(sentence), axis=-1))[0]
    proba =  np.max(model.predict(sentence))
    print(f"{result} : {proba}\n\n")

In [17]:
# Define your input text
sentences = [
            "He's over the moon about being accepted to the university",
            "Your point on this certain matter made me outrageous, how can you say so? This is insane.",
            "I can't do it, I'm not ready to lose anything, just leave me alone",
            "Merlin's beard harry, you can cast the Patronus charm! I'm amazed!"
            ]

for text in sentences:
    predict(text)

He's over the moon about being accepted to the university
1/1 [==============================] - 0s 160ms/step
joy : 0.9486452341079712


Your point on this certain matter made me outrageous, how can you say so? This is insane.
1/1 [==============================] - 0s 157ms/step
anger : 0.5746949315071106


I can't do it, I'm not ready to lose anything, just leave me alone
1/1 [==============================] - 0s 162ms/step
anger : 0.6555194854736328


Merlin's beard harry, you can cast the Patronus charm! I'm amazed!
1/1 [==============================] - 0s 159ms/step
surprise : 0.8913914561271667




In [18]:
sentence= 'my old brother is dead'
print(sentence)
sentence = normalized_sentence(sentence)
sentence = tokenizer.texts_to_sequences([sentence])
sentence = pad_sequences(sentence, maxlen=229, truncating='pre')
result = le.inverse_transform(np.argmax(model.predict(sentence), axis=-1))[0]
proba =  np.max(model.predict(sentence))
print(f"{result} : {proba}\n\n")

my old brother is dead
1/1 [==============================] - 0s 165ms/step
sadness : 0.9940859079360962


